## 调用chatgpt api方式启动

In [14]:
from langchain.schema import HumanMessage,SystemMessage
from langchain_openai import ChatOpenAI,OpenAI
from langchain.prompts import ChatPromptTemplate,HumanMessagePromptTemplate,SystemMessagePromptTemplate

import os
os.environ["OPENAI_API_KEY"] = ""


#### ChatOpenAi

In [30]:
chat=ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    temperature=0.5,
)
message=[
    SystemMessage(content="you are a chatbot."),
    HumanMessage(content="Hello,introduce youself"),
]
res=chat(message)
print(res.content)

Hello! I am a chatbot here to assist you with any questions or tasks you may have. Feel free to ask me anything!


#### OpenAi

In [32]:
from langchain_openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0, max_tokens=512)

res=llm.stream("使用python编写快速排序算法。")
for chunk in res:
    print(chunk, end="", flush=True)



def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[0]
        left = [x for x in arr[1:] if x <= pivot]
        right = [x for x in arr[1:] if x > pivot]
        return quick_sort(left) + [pivot] + quick_sort(right)

# 测试
print(quick_sort([3, 2, 1, 5, 4]))  # 输出 [1, 2, 3, 4, 5]